In [1]:
import numpy
import numpy as np
import math
import pandas as pd
from scipy.optimize import minimize
import itertools
from functools import reduce
from gt_design import *

## Define Ranges

Define the range for the stage loading coefficient, flow coefficient and the reaction. Using the obtained values, calculate the other values.

Use the following data:
1. Reaction ($\Lambda$) -> 0.35 - 0.65
2. Flow Coefficient ($\phi$) -> 0.6 - 1.0
3. Structural Limit ($AN^2$) -> 10000000 - 22580600 (checking for a wide range)
4. Zweifel for vane -> 0.75 - 0.90
5. Zweifel for rotor -> 0.80 - 0.95

In [2]:
range_stage_loading = numpy.linspace(2.5, 4.5, 15)
range_alpha_3 = numpy.linspace(10, 25, 10)
range_mach_exit = numpy.linspace(0.3, 0.55, 10)
range_AN_squared = numpy.linspace(15000000, 22580600, 20)
range_zweifel_vane = numpy.linspace(0.75, 0.90, 4)
range_zweifel_rotor = numpy.linspace(0.80, 0.95, 4)
range_incidence = numpy.linspace(2, 4, 6)


## int main(){}

In [3]:
def int_main(range_mach_exit, range_stage_loading, range_alpha_3, range_AN_squared, range_incidence, range_zweifel_vane, range_zweifel_rotor):
    """
    Design Point Angles:
    beta_2m -> rotor inlet metal angle
    beta_3m -> rotor exit metal angle

    Off Design Angles:
    beta_2 -> rotor inlet flow angle
    beta_3 -> rotor exit flow angle

    IMP -> FOR DESIGN POINT -> beta_2m = beta_2, beta_3m -> beta_3

    INCIDENCE -> incidence = beta_2 - beta_2m, DEVIATION (for the project) -> beta_3 = beta_3m
    """
    data_meanline = []
    data_root_hub = []
    data_meanline_losses = []
    data_off_design = []
    data_efficiency = []
    data_scratch = []

    #for i, j, k, l, m, n, o, p in itertools.product(range_mach_exit, range_stage_loading, range_alpha_3, range_degree_reaction, range_AN_squared, range_incidence, range_zweifel_vane, range_zweifel_rotor):
    for i, j, k, m, n, o, p in itertools.product(range_mach_exit, range_stage_loading, range_alpha_3, range_AN_squared, range_incidence, range_zweifel_vane, range_zweifel_rotor):
        incidence = n

        # MEANLINE ANALYSIS
        T_1, P_1, rho_1, C_1 = aeroturbine.calc_properties(M_1, T_01, P_01)
        C_a_1 = C_1 * np.cos(np.radians(-10))
        T_3, P_3, rho_3, C_3 = aeroturbine.calc_properties(i, T_03_cooled, P_03)
        C_w_1 = numpy.sqrt(C_1**2 - C_a_1**2)
        
        # Calculate U from the stage loading.
        U = aeroturbine.calc_U(j)
        C_a_3, C_w_3,  V_3, V_w_3, flow_coefficient_3, beta_3m, a_3, M_3_rel, A_3,P_03_rel = aeroturbine.calc_stage_3(U, C_3, T_3, rho_3,P_3,k)
        T_02,T_2, P_2, rho_2, A_2, C_a_2, flow_coefficient_2, a_2, V_w_2, beta_2m, V_2, C_w_2, C_2, alpha_2, M_2, M_2_rel,P_02, P_02_rel,l = aeroturbine.calc_stage_2_trial(U, T_1, T_3, P_3, A_3, V_w_3)
        if T_02 > 0:

            # STRUCTURAL ANALYSIS
            A_1 = m_dot_1/(rho_1 * C_a_1)
            N, omega, r_hub, r_tip, r_meanline, h = aerostructural.calc_structural(m, A_2, U)

            if V_3 > V_2 and C_2 > C_1 and rho_3 < rho_2 < rho_1 and T_02 < T_01 and T_02 > T_03 and T_1 > T_2 and T_2 > T_3 and P_1 > P_2 and P_2 > P_3:

                # BLADE VORTEX ANALYSIS
                alpha_2_hub, alpha_3_hub, beta_2_hub, beta_3_hub, U_hub, V_2_hub, C_2_hub, M_2_rel_hub, M_2_hub, reaction_hub = aeroturbine.calc_hub_angles(r_meanline, r_hub, alpha_2, k, flow_coefficient_2, flow_coefficient_3, U, C_a_2, a_2,T_02,T_03,T_1,C_a_3)
                alpha_2_tip, alpha_3_tip, beta_2_tip, beta_3_tip, U_tip, V_2_tip, C_2_tip, M_2_rel_tip, M_2_tip, M_3_rel_tip = aeroturbine.calc_tip_angles(r_meanline, r_tip, alpha_2, k, flow_coefficient_2, flow_coefficient_3, U, C_a_2, a_2, T_02, C_a_3)
                #incerased alpha_2 to 73 from 70
                if 40 < alpha_2 < 75  and U_hub < 335.28  and reaction_hub > 0 and M_2_rel_hub > M_2_rel_tip and 3150 < omega < 4500 and P_03_rel < P_02_rel  and M_3_rel_tip < 1 and P_03 < P_02:
                    # ANGLES CHECK
                    if 0 < alpha_2_tip < alpha_2 < alpha_2_hub and 0 < beta_2_tip < beta_2m < beta_2_hub and 0 < alpha_3_tip < k < alpha_3_hub and beta_3_tip > beta_3m > beta_3_hub > 0 and M_2_hub < 1: 

                        # OFF DESIGN CALCULATIONS
                        T_3_od,rho_3_od, P_3_od, alpha_3_od, beta_2_od, flow_coeff_2_od, incidence_2, v_2_od, C_w_3_od,C_a_3_od,U_mean_od,flow_coeff_3_od, work_od_cw, work_od_vw, M_2_rel_od, M_3_rel_od, P_02_rel_od, P_03_rel_od = off_design.calc_off_design(A_3,U, beta_3m,C_a_2,C_w_2,beta_2m, a_2, a_3)

                        if flow_coeff_3_od > flow_coeff_2_od and C_a_3_od > 0 and 10 <= alpha_3_od <= 40 and P_02_rel_od > P_03_rel_od:

                            # VANE AREA AND HEIGHT
                            A_vane_mean = (A_1 + A_2)/2
                            h_vane_mean = (A_vane_mean * N/60)/U
                            r_tip_stator = r_meanline + (h_vane_mean/2)
                            r_hub_stator = r_meanline - (h_vane_mean/2)

                            # DESIGN POINT LOSSES
                            # CALCULATE LOSSES - Stator
                            K_p_stator, pitch_chord_ratio_stator, K_accel_stator, stagger_angle_stator, pitch_chord_ratio_stator, pitch_axial_chord_ratio_stator, K_1_s, K_2_s =  aerodynamic_losses.profile_losses.calc_K_p(M_1, M_2, M_2_hub, P_1, P_2, r_tip_stator, r_hub_stator, -10, alpha_2, o)
                            K_s_stator = aerodynamic_losses.secondary_losses.calc_K_s(K_accel_stator, AR_vane, -10, alpha_2)
                            K_TET_stator, N_stator, c_true_stator, c_a_stator,throat_opening_stator = aerodynamic_losses.trailing_edge_losses_stator.K_TET(M_1, -10, alpha_2, h_vane_mean, stagger_angle_stator, r_meanline, pitch_axial_chord_ratio_stator)
                            K_stator = K_p_stator + K_s_stator + K_TET_stator
                            pitch_stator = pitch_chord_ratio_stator*c_true_stator
                            
                            # CALCULATE LOSSES - Rotor
                            K_p_rotor, pitch_chord_ratio_rotor, K_accel_rotor, stagger_angle_rotor, pitch_chord_ratio_rotor, pitch_axial_chord_ratio_rotor, K_1_rotor, K_2_rotor =  aerodynamic_losses.profile_losses.calc_K_p(M_2_rel, M_3_rel, M_2_rel_hub, P_2, P_3, r_tip, r_hub,  beta_2m, beta_3m, p)
                            K_s_rotor = aerodynamic_losses.secondary_losses.calc_K_s(K_accel_rotor, AR_rotor, beta_2m, beta_3m)
                            K_TET_rotor, N_rotor, c_true_rotor, c_a_rotor, throat_opening_rotor = aerodynamic_losses.trailing_edge_losses_rotor.K_TET(M_2_rel, beta_2m, beta_3m, h, stagger_angle_rotor, r_meanline, pitch_axial_chord_ratio_rotor)
                            K_rotor = K_p_rotor + K_s_rotor + K_TET_rotor
                            pitch_rotor = pitch_chord_ratio_rotor*c_true_rotor

                            # EFFICIENCY CALCULATIONS DESIGN POINT
                            eta_tt = aerodynamic_losses.efficiency_calculations(K_stator, K_rotor, M_2, M_3_rel, C_2, V_3)
                            delta_n, eta_final = aerodynamic_losses.efficiency_final(eta_tt, h, beta_3m, r_tip, r_meanline)

                            # OFF DESIGN LOSSES
                            eta_tt_od, delta_n_od, eta_final_od = aerodynamic_losses.losses_off_design(K_p_rotor, K_s_rotor, K_stator, K_1_rotor, K_2_rotor, pitch_chord_ratio_rotor, pitch_axial_chord_ratio_rotor, c_true_rotor, beta_2m, beta_3m, incidence_2,  M_2_rel_od, M_3_rel_od, h, stagger_angle_rotor, r_meanline, r_tip, C_2, V_3)

                            # EFFICIENCY CALCULATIONS OFF DESIGN
                            zxs = verify_zweifel_stator(r_hub_stator, h_vane_mean, N_stator, c_a_stator ,-10, alpha_2)
                            zxr = verify_zweifel_rotor(r_hub, h, N_rotor, c_a_rotor , beta_2m, beta_3m)

                            # WORK CHECKS
                            work_Part_A = c_p_gas * 1000 * (T_01-T_03)
                            work_check_cw = U * (C_w_2 + C_w_3)
                            work_check_vw = U * (V_w_2 + V_w_3)

                            if eta_final_od > 0 and zxs < 0.89 and zxr < 0.94: # This is to filter the abnormal behavior with the graph    

                                # ===============================================   
                                # START OF LOOP 2
                                # THIS LOOP PROVIDES A NEW DESIGN POINT AND ALSO UPDATES THE VALUES FOR ROTOR INLET AND OUTLET RELATIVE FLOW ANGLES.
                                # Change the incidence angle
                                beta_2 = -1*incidence + beta_2m
                                beta_3 = beta_3m
                                incidence_off = incidence_2 - incidence

                                # OFF DESIGN LOSSES
                                eta_tt_od_new, delta_n_od_new, eta_final_od_new = aerodynamic_losses.losses_off_design(K_p_rotor, K_s_rotor, K_stator, K_1_rotor, K_2_rotor, pitch_chord_ratio_rotor, pitch_axial_chord_ratio_rotor, c_true_rotor, beta_2, beta_3, incidence_off,  M_2_rel_od, M_3_rel_od, h, stagger_angle_rotor, r_meanline, r_tip, C_2, V_3)

                                if eta_final_od_new > 0 and N_rotor < 50 and eta_final_od_new > 83:
                                    
                                    eta_final = eta_final/100
                                    eta_final_opt = 1/eta_final
                                    eta_final_od = eta_final_od/100
                                    eta_final_od_new = eta_final_od_new/100

                                    delta_eta_optimize = eta_final - eta_final_od_new
                                    data_scratch.append((zxs, zxr))
                                    data_meanline.append((j, l ,flow_coefficient_2, flow_coefficient_3, omega, m,    # Non dimensional parameters.
                                                        T_01, T_1, P_01, P_1, rho_1, T_02, T_2, P_02, P_02_rel, P_2, rho_2, T_03, T_3, P_03, P_03_rel, P_3, rho_3, A_1, A_2, A_3,     # Temperature and Pressure at different stations
                                                        M_1, C_1, C_a_1, C_w_1, -10, M_2, C_2, C_a_2,  C_w_2, alpha_2, M_2_rel, V_2, V_w_2, beta_2m, U, i, C_3, C_a_3, C_w_3, k, M_3_rel, V_3, V_w_3, beta_3m,work_Part_A,work_check_cw,work_check_vw))        # Triangles at different stations.
                                    # alpha_2_hub, alpha_3_hub, beta_2_hub, beta_3_hub, U_hub, V_2_hub, C_2_hub, M_2_rel_hub, M_2_hub, reaction_hub
                                    # alpha_2_tip, alpha_3_tip, beta_2_tip, beta_3_tip, U_tip, V_2_tip, C_2_tip, M_2_rel_tip, M_2_tip, M_3_rel_tip 
                                    data_root_hub.append((reaction_hub, h, r_tip, U_tip, r_hub, r_tip_stator, r_hub_stator, h_vane_mean,
                                                      alpha_2_hub, alpha_3_hub, beta_2_hub, beta_3_hub, U_hub, V_2_hub, C_2_hub, M_2_rel_hub, M_2_hub, reaction_hub,
                                                      alpha_2_tip, alpha_3_tip, beta_2_tip, beta_3_tip, U_tip, V_2_tip, C_2_tip, M_2_rel_tip, M_2_tip,M_3_rel_tip))

                                    data_meanline_losses.append((c_true_stator, c_a_stator, stagger_angle_stator,K_p_stator, K_s_stator, K_TET_stator, K_stator, c_true_rotor, c_a_rotor, stagger_angle_rotor, K_p_rotor, K_s_rotor, K_TET_rotor, K_rotor, pitch_stator, o,N_stator, pitch_rotor, p,N_rotor,throat_opening_stator,throat_opening_rotor))

                                    data_off_design.append((T_3_od, rho_3_od, P_3_od, alpha_3_od, beta_2_od, flow_coeff_2_od, incidence_2, incidence_off,v_2_od, C_w_3_od, C_a_3_od, U_mean_od, flow_coeff_3_od, work_od_cw, work_od_vw, M_2_rel_od, M_3_rel_od))

                                    data_efficiency.append((eta_tt, eta_final, eta_final_od, eta_final_od_new, eta_final_opt, delta_eta_optimize))


                                
    data_scratch = pd.DataFrame(data_scratch, columns=['zweifel_stator_NEW', 'zweifel_rotor_NEW'])                         
    data_meanline = pd.DataFrame(data_meanline, columns=['stage_loading', 'reaction_meanline', 'flow_coefficient_2', 'flow_coefficient_3', 'omega', 'AN_squared', 'T_01', 'T_1', 'P_01', 'P_1', 'rho_1', 'T_02', 'T_2', 'P_02', 'P_02_rel', 'P_2', 'rho_2', 'T_03', 'T_3', 'P_03', 'P_03_rel', 'P_3', 'rho_3', 'A_1', 'A_2', 'A_3', 'M_1', 'C_1', 'C_a_1', 'C_w_1', 'dummy', 'M_2', 'C_2', 'C_a_2', 'C_w_2', 'alpha_2', 'M_2_rel', 'V_2', 'V_w_2', 'beta_2m', 'U', 'i', 'C_3', 'C_a_3', 'C_w_3', 'k', 'M_3_rel', 'V_3', 'V_w_3', 'beta_3m','work_Part_A','work_check_cw','work_check_vw'])
    data_root_hub = pd.DataFrame(data_root_hub, columns=['reaction_hub', 'h', 'r_tip', 'U_tip','r_hub', 'r_tip_stator', 'r_hub_stator', 'h_vane_mean',
                                                         'alpha_2_hub', 'alpha_3_hub', 'beta_2_hub', 'beta_3_hub', 'U_hub', 'V_2_hub', 'C_2_hub', 'M_2_rel_hub', 'M_2_hub', 'reaction_hub',
                                                         'alpha_2_tip', 'alpha_3_tip', 'beta_2_tip', 'beta_3_tip', 'U_tip', 'V_2_tip', 'C_2_tip', 'M_2_rel_tip', 'M_2_tip', 'M_3_rel_tip'])
    data_meanline_losses = pd.DataFrame(data_meanline_losses, columns=['c_true_stator', 'c_a_stator','stagger_angle_stator','K_p_stator', 'K_s_stator', 'K_TET_stator', 'K_stator', 'c_true_rotor', 'c_a_rotor', 'stagger_angle_rotor','K_p_rotor', 'K_s_rotor', 'K_TET_rotor', 'K_rotor', 'pitch_stator', 'zweifel_stator','N_stator', 'pitch_rotor', 'zweifel_rotor','N_rotor','throat_opening_stator','throat_opening_rotor'])
    data_off_design = pd.DataFrame(data_off_design, columns=['T_3_od', 'rho_3_od', 'P_3_od', 'alpha_3_od', 'beta_2_od', 'flow_coeff_2_od', 'incidence_2', 'incidence_off', 'v_2_od', 'C_w_3_od', 'C_a_3_od', 'U_mean_od', 'flow_coeff_3_od', 'work_od_cw', 'work_od_vw', 'M_2_rel_od', 'M_3_rel_od'])
    data_efficiency = pd.DataFrame(data_efficiency, columns=['eta_tt', 'eta_final', 'eta_final_od', 'eta_final_od_new', 'eta_final_opt','delta_eta_optimize'])

    return data_meanline, data_root_hub, data_meanline_losses, data_off_design, data_efficiency, data_scratch


run int main

In [4]:
data_meanline, data_root_hub, data_meanline_losses, data_off_design, data_efficiency, data_scratch = int_main(range_mach_exit, range_stage_loading, range_alpha_3, range_AN_squared, range_incidence, range_zweifel_vane, range_zweifel_rotor)

### Define Optimization Parameters

In [5]:
data_efficiency['delta_eta_optimize_normalized'] = (data_efficiency['delta_eta_optimize'] - data_efficiency['delta_eta_optimize'].min()) / (data_efficiency['delta_eta_optimize'].max() - data_efficiency['delta_eta_optimize'].min())
data_meanline_losses['N_rotor_normalized'] = (data_meanline_losses['N_rotor'] - data_meanline_losses['N_rotor'].min()) / (data_meanline_losses['N_rotor'].max() - data_meanline_losses['N_rotor'].min())
data_meanline['AN_squared_normalized'] = (data_meanline['AN_squared'] - data_meanline['AN_squared'].min()) / (data_meanline['AN_squared'].max() - data_meanline['AN_squared'].min())
data_efficiency['eta_opt'] = 1/data_efficiency['eta_final']
data_efficiency['eta_opt_normalized'] = (data_efficiency['eta_opt'] - data_efficiency['eta_opt'].min()) / (data_efficiency['eta_opt'].max() - data_efficiency['eta_opt'].min())

In [6]:
# OPTIMIZATION FUNCTION
data_efficiency['func_optimize'] = 0.4 * (data_efficiency['eta_opt_normalized']) + 0.3 * (data_meanline_losses['N_rotor_normalized']) + 0.2 * (data_efficiency['delta_eta_optimize_normalized']) #+ 0.1 * data_meanline['AN_squared_normalized']

Write results to file

In [7]:
data_meanline.to_csv('_outputs/output_data_meanline.csv', index=False)
data_root_hub.to_csv('_outputs/output_data_root_hub.csv', index=False)
data_meanline_losses.to_csv('_outputs/output_data_meanline_losses.csv', index=False)
data_off_design.to_csv('_outputs/output_data_off_design.csv', index=False)
data_efficiency.to_csv('_outputs/output_data_efficiency.csv', index=False)
data_scratch.to_csv('_outputs/output_data_scratch.csv', index=False)